**Pendahuluan**
- dalam proyek in fokus utama adalah cleaning atau preparation dataset pada transaksi yang sukses ( seperti penanganan missing value dan sebagainya)

- fokus utama dataset ialah
1.  olist_orders_dataset.csv -> Data Master Pesanan (Tanggal, Status, ID)
2. olist_order_items_dataset.csv -> Detail produk di setiap pesanan
3. olist_products_dataset.csv -> Detail Produk (Nama Kategori, ID)
4. product_category_name_translation.csv -> Kamus Terjemahan Kategori (Português $\rightarrow$ English)

-----

**import dataset**

In [27]:
import pandas as pd

# dataPesanan
df_orders = pd.read_csv('olist_orders_dataset.csv')
# dataDetailProduk_setiap pesanan
df_orders_items = pd.read_csv('olist_order_items_dataset.csv')


# Lihat 5 baris pertama_orders_Pesanan
print(df_orders.head())

                           order_id                       customer_id  \
0  e481f51cbdc54678b7cc49136f2d6af7  9ef432eb6251297304e76186b10a928d   
1  53cdb2fc8bc7dce0b6741e2150273451  b0830fb4747a6c6d20dea0b8c802d7ef   
2  47770eb9100c2d0c44946d9cf07ec65d  41ce2a54c0b03bf3443c3d931a367089   
3  949d5b44dbf5de918fe9c16f97b45f8a  f88197465ea7920adcdbec7375364d82   
4  ad21c59c0840e6cb83a9ceb5573f8159  8ab97904e6daea8866dbdbc4fb7aad2c   

  order_status order_purchase_timestamp    order_approved_at  \
0    delivered      2017-10-02 10:56:33  2017-10-02 11:07:15   
1    delivered      2018-07-24 20:41:37  2018-07-26 03:24:27   
2    delivered      2018-08-08 08:38:49  2018-08-08 08:55:23   
3    delivered      2017-11-18 19:28:06  2017-11-18 19:45:59   
4    delivered      2018-02-13 21:18:39  2018-02-13 22:20:29   

  order_delivered_carrier_date order_delivered_customer_date  \
0          2017-10-04 19:55:00           2017-10-10 21:25:13   
1          2018-07-26 14:31:00           2018-08

----

**merge table untuk memperkaya insight**

In [23]:
# Gabungkan Orders (Kiri) dengan Order Items (Kanan)
# Gunakan how='left' untuk melihat apakah ada order yang tidak punya item
df_merged = df_orders.merge(df_orders_items, on='order_id', how='left')

# Cek apakah ada "Kotoran" (Order tanpa item)
if df_merged['product_id'].isnull().sum() > 0:
    print("Ada order tanpa item. Periksa data lebih lanjut. we got first dirty data")
else:
    print("Semua order punya item. Data bersih dari order tanpa item.")
 
print('=='*40)
print("Jumlah Order tanpa Item:", df_merged['product_id'].isnull().sum())
print('=='*40)
print(df_merged['order_status'].unique())

Ada order tanpa item. Periksa data lebih lanjut. we got first dirty data
Jumlah Order tanpa Item: 775
['delivered' 'invoiced' 'shipped' 'processing' 'unavailable' 'canceled'
 'created' 'approved']


----

**Hitung jumlah statusnya dan melihat distribusinya**

In [24]:
# Ambil hanya data yang 'product_id'-nya kosong (NaN)
df_orders_no_items = df_merged[df_merged['product_id'].isnull()]

# Hitung jumlah statusnya dan melihat distribusinya
print(df_orders_no_items['order_status'].value_counts())

order_status
unavailable    603
canceled       164
created          5
invoiced         2
shipped          1
Name: count, dtype: int64


---
**(Drop Rows)**

- an insight -> tujuan kita biasanya ingin menganalisis "Penjualan" (Revenue/Sales), order yang tidak ada barangnya (artinya tidak ada harganya) adalah SAMPAH atau hanya mengambil data yang benar2 terjual.

In [25]:
# RINGKAS ambil yang benar benar delivered (Whitelisting)
# Daripada membuang sampah satu-satu, ambil saja berliannya.
df_clean_final = df_merged[df_merged['order_status'] == 'delivered']

print(df_clean_final['order_status'].value_counts())

print('=='*40)

print(df_clean_final.head(5))

order_status
delivered    110197
Name: count, dtype: int64
                           order_id                       customer_id  \
0  e481f51cbdc54678b7cc49136f2d6af7  9ef432eb6251297304e76186b10a928d   
1  53cdb2fc8bc7dce0b6741e2150273451  b0830fb4747a6c6d20dea0b8c802d7ef   
2  47770eb9100c2d0c44946d9cf07ec65d  41ce2a54c0b03bf3443c3d931a367089   
3  949d5b44dbf5de918fe9c16f97b45f8a  f88197465ea7920adcdbec7375364d82   
4  ad21c59c0840e6cb83a9ceb5573f8159  8ab97904e6daea8866dbdbc4fb7aad2c   

  order_status order_purchase_timestamp    order_approved_at  \
0    delivered      2017-10-02 10:56:33  2017-10-02 11:07:15   
1    delivered      2018-07-24 20:41:37  2018-07-26 03:24:27   
2    delivered      2018-08-08 08:38:49  2018-08-08 08:55:23   
3    delivered      2017-11-18 19:28:06  2017-11-18 19:45:59   
4    delivered      2018-02-13 21:18:39  2018-02-13 22:20:29   

  order_delivered_carrier_date order_delivered_customer_date  \
0          2017-10-04 19:55:00           2017-10-10 2

----
**Double Merge untuk mengubah kode product_id $\rightarrow$ Nama Portugis $\rightarrow$ Nama Inggris.**

In [30]:
# dataProduk
df_products = pd.read_csv('olist_products_dataset.csv')

print(df_products.head())


                         product_id  product_category_name  \
0  1e9e8ef04dbcff4541ed26657ea517e5             perfumaria   
1  3aa071139cb16b67ca9e5dea641aaa2f                  artes   
2  96bd76ec8810374ed1b65e291975717f          esporte_lazer   
3  cef67bcfe19066a932b7673e239eb23d                  bebes   
4  9dc1a7de274444849c219cff195d0b71  utilidades_domesticas   

   product_name_lenght  product_description_lenght  product_photos_qty  \
0                 40.0                       287.0                 1.0   
1                 44.0                       276.0                 1.0   
2                 46.0                       250.0                 1.0   
3                 27.0                       261.0                 1.0   
4                 37.0                       402.0                 4.0   

   product_weight_g  product_length_cm  product_height_cm  product_width_cm  
0             225.0               16.0               10.0              14.0  
1            1000.0       

In [40]:
# Tempelkan Nama Portugis ke Data Transaksi
# merge ke df_clean_final yang sudah di whitelis
df_step1 = pd.merge(df_clean_final, df_products, on='product_id', how='left')
print(df_step1[['order_id', 'order_status', 'product_category_name']].head())


                           order_id order_status  product_category_name
0  e481f51cbdc54678b7cc49136f2d6af7    delivered  utilidades_domesticas
1  53cdb2fc8bc7dce0b6741e2150273451    delivered             perfumaria
2  47770eb9100c2d0c44946d9cf07ec65d    delivered             automotivo
3  949d5b44dbf5de918fe9c16f97b45f8a    delivered               pet_shop
4  ad21c59c0840e6cb83a9ceb5573f8159    delivered              papelaria


In [38]:
# dataKamusKategoriProduk
df_product_category_name_translation = pd.read_csv('product_category_name_translation.csv')
print(df_product_category_name_translation.head())


    product_category_name product_category_name_english
0            beleza_saude                 health_beauty
1  informatica_acessorios         computers_accessories
2              automotivo                          auto
3         cama_mesa_banho                bed_bath_table
4        moveis_decoracao               furniture_decor


In [42]:
# Tempelkan Nama Inggris (Translate)
# (Merge berdasarkan kolom 'product_category_name' yang bahasa Portugis)
df_master = pd.merge(df_step1, df_product_category_name_translation, on='product_category_name', how='left')
df_master[['order_id', 'product_category_name', 'product_category_name_english']].head()

,order_id,product_category_name,product_category_name_english
0,e481f51cbdc54678b7cc49136f2d6af7,utilidades_domesticas,housewares
1,53cdb2fc8bc7dce0b6741e2150273451,perfumaria,perfumery
2,47770eb9100c2d0c44946d9cf07ec65d,automotivo,auto
3,949d5b44dbf5de918fe9c16f97b45f8a,pet_shop,pet_shop
4,ad21c59c0840e6cb83a9ceb5573f8159,papelaria,stationery


In [43]:
# Buang kolom yang tidak perlu
# hanya butuh 'product_category_name_english', membuang portugis supaya rapih .
df_master_clean = df_master.drop(columns=['product_category_name'])
df_master_clean = df_master_clean.rename(columns={'product_category_name_english': 'category'})

# LIHAT HASILNYA!
print(df_master_clean[['order_id', 'price', 'category']].head())

                           order_id   price    category
0  e481f51cbdc54678b7cc49136f2d6af7   29.99  housewares
1  53cdb2fc8bc7dce0b6741e2150273451  118.70   perfumery
2  47770eb9100c2d0c44946d9cf07ec65d  159.90        auto
3  949d5b44dbf5de918fe9c16f97b45f8a   45.00    pet_shop
4  ad21c59c0840e6cb83a9ceb5573f8159   19.90  stationery


In [44]:
df_master_clean.head()

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,order_item_id,product_id,...,price,freight_value,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm,category
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00,1.0,87285b34884572647811a353c7ac498a,...,29.99,8.72,40.0,268.0,4.0,500.0,19.0,8.0,13.0,housewares
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13 00:00:00,1.0,595fac2a385ac33a80bd5114aec74eb8,...,118.70,22.76,29.0,178.0,1.0,400.0,19.0,13.0,19.0,perfumery
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04 00:00:00,1.0,aa4383b373c6aca5d8797843e5594415,...,159.90,19.22,46.0,232.0,1.0,420.0,24.0,19.0,21.0,auto
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15 00:00:00,1.0,d0b61bfb1de832b15ba9d266ca96e5b0,...,45.00,27.20,59.0,468.0,3.0,450.0,30.0,10.0,20.0,pet_shop
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26 00:00:00,1.0,65266b2da20d04dbe00c5c2d3bb7859e,...,19.90,8.72,38.0,316.0,4.0,250.0,51.0,15.0,15.0,stationery


---
**Checkpoint (Cek & Ricek)**

In [45]:
# 1. Cek apakah ada produk yang GAGAL diterjemahkan?
# (Kadang ada nama portugis yang gak ada di kamus translate, jadi NaN)
jumlah_gagal = df_master_clean['category'].isnull().sum()
print(f"Kategori yang masih kosong (NaN): {jumlah_gagal}")

# 2. Cek Tipe Data Tanggal
# Penting: Kolom waktu seringkali dianggap Teks (Object) oleh Python
print("\nTipe Data Sebelum Diubah:")
print(df_master_clean['order_purchase_timestamp'].dtype)

Kategori yang masih kosong (NaN): 1559

Tipe Data Sebelum Diubah:
object


In [46]:
# Tambal Kategori Kosong
# isi dengan 'others' supaya aman
df_master_clean['category'] = df_master_clean['category'].fillna('others')

# 2. Ubah Teks Tanggal jadi Format Waktu (Datetime)
df_master_clean['order_purchase_timestamp'] = pd.to_datetime(df_master_clean['order_purchase_timestamp'])

# --- CEK HASIL AKHIR ---
print("Sisa Kategori Kosong:", df_master_clean['category'].isnull().sum())
print("Tipe Data Tanggal:", df_master_clean['order_purchase_timestamp'].dtype)

Sisa Kategori Kosong: 0
Tipe Data Tanggal: datetime64[ns]


In [48]:
df_master_clean.to_csv('data_clean_siap_pakai_brazilian_e.csv', index=False)